In [250]:
dice(x) = (x[1] - 1)^2 + sum(i * (2 * x[i]^2 - x[i-1])^2 for i in 2:length(x))

dice (generic function with 1 method)

In [251]:
# x = 0:0.1:10
# y = 0:0.1:10

# # Create a grid of (x, y) values
# X = [x_val for x_val in x, y_val in y]
# Y = [y_val for x_val in x, y_val in y]

# # Calculate the dice values for the grid
# Z = [dice([x_val, y_val]) for x_val in x, y_val in y]
# using Plots

# plot(
#     x, y, Z, st=:surface, legend=false, c=:viridis,
#     xlabel="X", ylabel="Y", zlabel="Dice(x, 2)"
# )

In [252]:
no_of_gen = 20
dimension = 3
chromosome = rand(-10:10, no_of_gen, dimension)

20×3 Matrix{Int64}:
 -2    7  -4
 -4   -6  -3
  1    6   9
 -1  -10  -4
  4   -6   1
 -5   -9  10
  2   -6   1
 -7    0   0
 10   -7  -2
 -6    7  -1
  3    2  -1
 -9   -3  -1
  5    0  -1
  5  -10  10
  8   -7   8
 -1    7  -3
 -4   -8   6
  2    6  -5
 -3    3   2
 -5   -1  -2

In [253]:
func_values = dice.(eachrow(chromosome))
sum(func_values)/length(func_values)

41044.7

In [254]:
fitness = 1 ./ (func_values .+ 1) # In order to avoid dividing them by zero

20-element Vector{Float64}:
 4.569339730408956e-5
 7.515406583496167e-5
 1.2034997773525411e-5
 1.1614536754201559e-5
 0.00010582010582010582
 5.35165740829935e-6
 0.00010006003602161297
 0.006134969325153374
 6.155740227762388e-5
 4.596221905593602e-5
 0.01818181818181818
 0.0006119951040391676
 0.012658227848101266
 4.799224445329635e-6
 1.4099400775467043e-5
 5.0075112669003505e-5
 1.849317601804934e-5
 6.406149903907751e-5
 0.001026694045174538
 0.0026455026455026454

In [255]:
indi_prob = fitness ./ sum(fitness)
cumul_prob = cumsum(indi_prob)

20-element Vector{Float64}:
 0.0010912120834940704
 0.0028859795904283654
 0.0031733894958004003
 0.003450758297380764
 0.005977866916139255
 0.006105670790992179
 0.008495222166544284
 0.15500550711910707
 0.15647557030453446
 0.15757320216771725
 0.5917764102998577
 0.6063915733642811
 0.9086849461145055
 0.9087995572402944
 0.909136267518014
 0.9103321201192793
 0.9107737589188367
 0.9123036228808654
 0.9368222845310377
 0.9999999999999999

In [256]:
roulette = rand(length(cumul_prob))

20-element Vector{Float64}:
 0.015709939223577662
 0.5606336559033461
 0.37024645520451394
 0.4364110254977288
 0.036114638092555995
 0.25947026260955597
 0.7111373183056834
 0.6937350550659676
 0.32679652318614905
 0.08725856966562617
 0.8860396805679966
 0.7607350296707822
 0.05451811162189635
 0.8711446841384041
 0.759252569378574
 0.5877401500049991
 0.6601423839549172
 0.8142501110524555
 0.24828892099791589
 0.5741099876890126

In [257]:
new_chromo = []
for i = 1:length(roulette)
    for j = 1:length(roulette)-1
        if roulette[i] > cumul_prob[j] && roulette[i] <= cumul_prob[j+1]
            push!(new_chromo, chromosome[j+1, :])
        end
    end
end
chromosome = mapreduce(permutedims, vcat, new_chromo)
#chromosome=hcat(new_chromo...)'

20×3 Matrix{Int64}:
 -7  0   0
  3  2  -1
  3  2  -1
  3  2  -1
 -7  0   0
  3  2  -1
  5  0  -1
  5  0  -1
  3  2  -1
 -7  0   0
  5  0  -1
  5  0  -1
 -7  0   0
  5  0  -1
  5  0  -1
  3  2  -1
  5  0  -1
  5  0  -1
  3  2  -1
  3  2  -1

## Crossover

In [258]:
crossover_rate = 0.3
parent_index = []
for i in 1:length(new_chromo)
    crossover_rand = rand()
    if crossover_rand<=crossover_rate
        push!(parent_index,i)
    end
end
parent_index

2-element Vector{Any}:
  5
 12

In [259]:
child=[]
for i in 1:length(parent_index)
    first_parent=chromosome[parent_index[i],:]
    second_parent = chromosome[rand((parent_index[parent_index .!= parent_index[i]])),:]
    crossover_point=rand(1:length(chromosome[1,:]) - 1)
    push!(child,[first_parent[1:crossover_point];second_parent[crossover_point+1:end]])
end
child

2-element Vector{Any}:
 [-7, 0, -1]
 [5, 0, 0]

In [260]:
for i in 1:length(parent_index)
    chromosome[i,:]= child[i]
end
chromosome

20×3 Matrix{Int64}:
 -7  0  -1
  5  0   0
  3  2  -1
  3  2  -1
 -7  0   0
  3  2  -1
  5  0  -1
  5  0  -1
  3  2  -1
 -7  0   0
  5  0  -1
  5  0  -1
 -7  0   0
  5  0  -1
  5  0  -1
  3  2  -1
  5  0  -1
  5  0  -1
  3  2  -1
  3  2  -1

## Mutation

In [261]:
mutation_rate = 0.06 # 6%
mutation_indices = rand(1:length(chromosome), round(Int, mutation_rate * length(chromosome)))

for index in unique(mutation_indices)
    chromosome[index] = rand(-10:10)
end
chromosome

20×3 Matrix{Int64}:
 -7   0   -1
  5   0    6
  3   2   -1
  3   2   -1
 -7   0    0
  3   2   -1
  5   0   -1
  5   0   -1
  3   2   -1
 -7   0    0
  5   0   -1
  5   0   -1
 -7   0    0
  5   0   -1
  5   0  -10
  1   2   -1
  5   0   -1
  5  -3   -1
  3   2   -1
  3   2   -1

In [262]:
func_values = dice.(eachrow(chromosome))
sum(func_values)/length(func_values)

6883.15